In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!git clone https://github.com/soroushdty/pdm.git

Cloning into 'pdm'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 34 (delta 6), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 19.62 KiB | 669.00 KiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
# Add project folder to PYTHONPATH
import os
import sys
from pathlib import Path

PROJECT_NAME = "pdm"
PROJECT_DIR = (Path.cwd() / PROJECT_NAME).resolve()
if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

os.chdir(PROJECT_DIR)

# Install requirements
from funcs.requirements_utils import install_missing
# Assuming requirements.txt is within the PROJECT_DIR
installed = install_missing(PROJECT_DIR / "requirements.txt", quiet=False)
print("Installed:", installed)

Installed: []


In [ ]:
# Load config
from funcs.config_loader import load_config, validate_config

CONFIG_PATH = os.path.join(PROJECT_DIR, "config.json")
cfg = validate_config(load_config(CONFIG_PATH))
cfg

{'dataset': '/content/drive/MyDrive/pdm/input/dataset.xlsx',
 'patient_col': 'Patient',
 'physician_col': 'Physician',
 'item_col': 'Item',
 'output_path': '/content/drive/MyDrive/pdm/output/1_dedup',
 'duplicates': '/content/drive/MyDrive/pdm/configs/duplicates.xlsx',
 'classes': ['Behavioral health',
  'Diagnoses',
  'Disabilities',
  'Infectious diseases',
  'Genetics',
  'Medications',
  'Sexual and reproductive health',
  'Social determinants of health',
  'Violence',
  'Other']}

In [ ]:
# Create run directory + logger, then run pipeline
from funcs.paths import make_run_dir
from funcs.logger_setup import setup_logger
from funcs.preprocessing import run
from pathlib import Path

run_dir = make_run_dir(cfg["output_path"])
logger = setup_logger(run_dir / "log.txt")

# IMPORTANT: pipeline writes outputs into cfg["output_path"].
# We'll point cfg output_path at this run directory for this run.
#note: resolve later
cfg_run = dict(cfg)
cfg_run["output_path"] = str(run_dir)

p = run(cfg_run)
train = p.paths.train_csv
test = p.paths.test_csv
train_map = p.paths.train_map
test_map = p.paths.test_map